In [ ]:
import polars as pl
import polars_ds as pds

In [ ]:
df = pds.random_data(size=20_000, n_cols=0).select(
    pds.random(0.0, 1.0).alias("x1"),
    pds.random(0.0, 1.0).alias("x2"),
    pds.random(0.0, 1.0).alias("x3"),
    pds.random(0.0, 1.0).alias("x4"),
    pds.random(0.0, 1.0).alias("x5"),
    pds.random_int(0, 3).cast(pl.String).alias("str"),
    pds.random_int(0, 100).alias("test"),
)
df

In [ ]:
# df = pl.DataFrame({
#     "a": list(range(100))
# })

import numpy as np

df = pl.DataFrame({
    "a": np.sin(2 * np.pi * np.arange(3000) / 100)
})

In [ ]:
df.select(
    pds.query_sample_entropy("a")
)

In [ ]:
import tracemalloc

In [ ]:
tracemalloc.start()
df.with_columns(
    pds.query_nb_cnt(
        0.05, # radius 
        "x1", "x2", "x3", "x4", "x5", 
        dist = "l2",
        parallel = True
    ).alias("nb_cnt")
)
print(tracemalloc.get_traced_memory())
tracemalloc.stop()

In [ ]:

df.with_columns(
    pds.query_nb_cnt(
        0.1, # radius 
        "x1", "x2", "x3", # "x4", "x5", 
        dist = "l2",
        parallel = True
    ).alias("nb_cnt")
)

In [ ]:
tracemalloc.start()
df.with_columns(
    pds.query_nb_cnt2(
        0.05, # radius 
        "x1", "x2", "x3", "x4", "x5", 
        dist = "l2",
        parallel = True 
    ).alias("nb_cnt")
)
print(tracemalloc.get_traced_memory())
tracemalloc.stop()

In [ ]:

df.with_columns(
    pds.query_nb_cnt2(
        0.1, # radius 
        "x1", "x2", "x3", # "x4", "x5", 
        dist = "l2",
        parallel = True 
    ).alias("nb_cnt")
)